In [1]:
from requests import get
from bs4 import BeautifulSoup
import json
import time
import os
import random
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
URL = "https://providence.craigslist.org/"
QUERY = "search/apa"

DTYPE= 'craigslist-v1'

## Providence only
#postal_codes = ['02903', '02904', '02905', '02906', '02907', '02908', '02909', '02910', '02911', '02919', ]

postal_codes = {
    "02801": "Adamsville, Newport County",
    "02802": "Albion, Providence County",
    "02804": "Ashaway, Washington County",
    "02806": "Barrington, Bristol County",
    "02807": "Block Island, New ShorehamWashington County",
    "02808": "Bradford, Washington County",
    "02809": "Bristol, Bristol Count",
    "02812": "Carolina, RichmondWashington County",
    "02813": "Charlestown, Washington County",
    "02814": "Chepachet, Providence County",
    "02815": "Clayville, Providence County",
    "02816": "Coventry, Kent County",
    "02817": "West Greenwich, W GreenwichKent County",
    "02818": "East Greenwich, E GreenwichKent County",
    "02822": "Exeter, EscoheagWashington County",
    "02823": "Fiskeville, Providence County",
    "02824": "Forestdale, Providence County",
    "02825": "Foster, Providence County",
    "02826": "Glendale, Providence County",
    "02827": "Greene, CoventryKent County",
    "02828": "Greenville, Providence County",
    "02829": "Harmony, Providence County",
    "02830": "Harrisville, Burrillville, Providence County",
    "02831": "HopeProvidence County",
    "02832": "Hope Valley, Richmond, Washington County",
    "02833": "Hopkinton, Washington County",
    "02835": "Jamestown, Newport County",
    "02836": "Kenyon, Richmond, Washington County",
    "02837": "Little Compton, L Compton, Newport County",
    "02838": "Manville, Providence County",
    "02839": "Mapleville, Providence County",
    "02840": "Newport, Newport County",
    "02841": "Newport, Newport County",
    "02842": "Middletown, Newport County",
    "02852": "North Kingstown, N Kingstown, Washington County",
    "02857": "North Scituate, N Scituate, Scituate, Providence County",
    "02858": "Oakland, Providence County",
    "02859": "Pascoag, Providence County",
    "02860": "Pawtucket, Providence County",
    "02861": "Pawtucket, Providence County",
    "02862": "Pawtucket, Providence County",
    "02863": "Central Falls, Providence County",
    "02864": "Cumberland, Providence County",
    "02865": "Lincoln, Providence County",
    "02871": "Portsmouth, Newport County",
    "02872": "Prudence Island, Prudence Isl, Newport County",
    "02873": "Rockville, Washington County",
    "02874": "Saunderstown, Washington County",
    "02875": "Shannock, Richmond, Washington County",
    "02876": "Slatersville, Providence County",
    "02877": "Slocum, Washington County",
    "02878": "Tiverton, Newport County",
    "02879": "Wakefield, Narragansett, Peace Dale, S Kingstown, S...Washington County",
    "02880": "Wakefield, Washington County",
    "02881": "Kingston, Washington County",
    "02882": "Narragansett, Point Judith, Washington County",
    "02883": "Peace Dale, S Kingstown, South Kingstown, Washington County",
    "02885": "Warren, Bristol County",
    "02886": "Warwick,Kent County",
    "02887": "Warwick,Kent County",
    "02888": "Warwick,Kent County",
    "02889": "Warwick,Kent County",
    "02891": "Westerly, Washington County",
    "02892": "West Kingston, Richmond, Washington County",
    "02893": "West Warwick, Kent County",
    "02894": "Wood River Junction, Wood River Jt, Washington County",
    "02895": "Woonsocket, Providence County",
    "02896": "North Smithfield, N Smithfield, Providence County",
    "02898": "Wyoming, Richmond, Washington County",
    "02901": "Providence,Providence County",
    "02902": "Providence,Providence County",
    "02903": "Providence,Providence County",
    "02904": "Providence, N Providence, North Providence,Providence County",
    "02905": "Providence, Cranston,Providence County",
    "02906": "Providence,Providence County",
    "02907": "Providence, Cranston,Providence County",
    "02908": "Providence, N Providence, North Providence,Providence County",
    "02909": "Providence,Providence County",
    "02910": "Cranston, Providence,Providence County",
    "02911": "North Providence, N Providence, Providence,Providence County",
    "02912": "Providence, Brown Station, Providence County",
    "02914": "East Providence, E Providence,Providence County",
    "02915": "Riverside,Providence County",
    "02916": "Rumford,Providence County",
    "02917": "Smithfield,Providence County",
    "02918": "Providence, Friar Station,Providence County",
    "02919": "Johnston, Providence,Providence County",
    "02920": "Cranston, Providence County",
    "02921": "Cranston, Providence County",
    "02940": "Providence, Providence County",
}


def query_url():
    return URL + QUERY

def filter_line(line):
    if not line or line == 'QR Code Link to This Post':
        return False
    else:
        return True

def map_line(line):
    return line.strip('\n').strip(' ')

def get_description(href):
    time.sleep(random.uniform(0.2, 0.8))
    response_details = get(href)
    detail_soup = BeautifulSoup(response_details.text, 'html.parser')
    body = detail_soup.find(id="postingbody")
    
    return list(filter(filter_line, map(map_line, body.find_all(text=True))))

def extract_repost(duplicate_rows):
    if duplicate_rows:
        return [row.get('data-repost-of') for row in duplicate_rows.find_all(class_="result-row")]
    else:
        return []

def listing(post):
    title = post.find(class_ = "result-title hdrlnk")
    print("Getting: {}".format(title.text))
    return {
        'price': post.find(class_ = "result-price").text, 
        'title': title.text,
        'href': title.get('href'),
        'description': get_description(title.get('href')),
        'duplicates': extract_repost(post.find(class_="duplicate-rows")),
    }


def extract_listings(postal_code, area):
    response = get(query_url(), params={'bundleDuplicates':1, 'availabilityMode':0, 'sale_date':'all dates', 'search_distance': 0, 'postal': postal_code})
    html_soup = BeautifulSoup(response.text, 'html.parser')
    posts = html_soup.find_all('li', class_= ['result-row', 'result-row duplicate-row'])
    dup_posts = html_soup.find_all('li', class_= 'result-row duplicate-row')

    print("Got type: {}; listings: {}; dup listings: {} for {}".format(type(posts), len(posts), len(dup_posts), postal_code))

    # Is there pagination? Seems to match UI well but not exactly?
    extracted = [listing(post) for post in posts + dup_posts]
   
    
    raw_path = os.path.join(DTYPE, 'raw_html')
    os.makedirs(raw_path, exist_ok=True)
    with open(os.path.join(raw_path, '{}-{}.html'.format(postal_code, area)), 'w') as f:
        f.write(response.text)

    extracted_path = os.path.join(DTYPE, 'extracted')
    os.makedirs(extracted_path, exist_ok=True)
    with open(os.path.join(extracted_path, '{}-{}.json'.format(postal_code, area)), 'w') as f:
        json.dump(extracted, f, indent=2)
        
#extract_listings('02905', 'foo_bar, baz')
        
for postal_code, area in postal_codes.items():
     extract_listings(postal_code, area)
        

Got type: <class 'bs4.element.ResultSet'>; listings: 2; dup listings: 0 for 02801
Getting: LOOKING FOR A GREAT PLACE TO LIVE?
Getting: Duplex 1 bedroom 1 bath apartment rent~~!@~
Got type: <class 'bs4.element.ResultSet'>; listings: 6; dup listings: 0 for 02802
Getting: 2 larger bedrooms with 2 closets
Getting: Albion/ Lincoln
Getting: Beautifully remodeled 2 Bedroom 1 1/2 Bathroom Condo
Getting: 3 BR Apartment For Rent
Getting: Bright Sunny open Layout Lincoln RI 2 Large Bedroom 2 Large Baths
Getting: Beautiful 1 Bedroom Condo - Furnished
Got type: <class 'bs4.element.ResultSet'>; listings: 2; dup listings: 0 for 02804
Getting: Ashaway Duplex for Rent
Getting: Gorgeous Riverfront Property
Got type: <class 'bs4.element.ResultSet'>; listings: 14; dup listings: 2 for 02806
Getting: Our Perfect Rental Is Waiting For You
Getting: Brand new 2 Bedroom Condo Available immediately in Barrington
Getting: Newer Construction. Haines Park Village SAMSON REALTY 5931
Getting: Modern Two-Story Townhou

Getting: Proud Supporter of Move for Hunger, Fully-Applianced Kitchen, Dog park
Getting: Natural Gas Heat, In Home Washer/Dryer (Full Size), Fitness Center
Getting: Natural Gas Heat, Security Deposit Alternative Available, Dog park
Getting: Professionally Managed by DOLBEN, Wall-to-wall carpet, Clubhouse
Getting: Disability Access, Proud Affiliate of the NAA, Online Payments
Getting: Dogs/Cats Accepted, In Home Washer/Dryer (Full Size), Fitness Center
Getting: Dogs/Cats Accepted, Wired for High Speed Internet, 24-hr Maintenance
Getting: Clubhouse, Online Payments, Swimming pool
Getting: Fully-Applianced Kitchen, Dog park, Professionally Managed by DOLBEN
Getting: Online Work Orders, Private Balcony or Patio, Disability Access
Getting: Online Work Orders, Professionally Managed by DOLBEN, Swimming pool
Getting: Natural Gas Heat, Cable Ready, Proud Supporter of Move for Hunger
Getting: 24-hr Maintenance, Renovated kitchens and bathrooms, Dog park
Getting: Security Deposit Alternative Ava

Getting: $500 security deposit with good credit on 1-2 beds Anthony Mill A-111
Getting: You are sure to love it. The Lofts at Anthony Mill!
Getting: Location, Community, Quality Living. It Starts Here!
Getting: Home is our favorite four-letter word.
Getting: The Lifestyle You Deserve Anthony Mill (A-111)
Getting: Unique Layout, Private Entrance.  Anthony Mill (B-103)
Getting: $500 Security Deposit With Good Credit On 1-2 Beds Anthony Mill A-310
Getting: “Woof. Woof. Meow Meow.” Don’t Worry, They Understand.
Getting: Where You Belong. The Lofts at Anthony Mill!
Getting: Anthony Mill Welcomes Your Home!(#103)
Getting: Modern Conveniences & Entertainment, All Within Your Reach (A-411)
Getting: Come Home With Confidence to Harris Mill Lofts (#1-401)
Getting: Anthony Mills Offers Spacious and Modern Living (#111)
Getting: Harris Mill Lofts - AVAILABILITY
Getting: Schedule a tour of a loft today!
Getting: New Special Going On At Harris Mill, Right Now!
Getting: The Lifestyle You Deserve Anth

Getting: Check Anthony Mill Out Today!
Getting: Spacious 2 Bed, Walk In Closet! Anthony Mill (A-109)
Getting: $500 security deposit with good credit on 1-2 beds Anthony Mill A-210
Getting: 2 Bedroom/2 Bathroom Pet Friendly Apartment
Getting: 2 Bedroom/1 Bathroom & Walk-In Closet #A109
Getting: Harris Mill Offers Beautiful Apartments! (1-405)
Getting: Unique Layout, Private Entrance.  Anthony Mill (B-103)
Getting: Spacious 2 Bed, Walk In Closet! Anthony Mill (A-109)
Getting: $500 security deposit with good credit on 1-2 beds Anthony Mill A-210
Getting: Harris Mill Lofts: A Higher Quality of Living (#3-215)
Getting: Lease Today Before Specials Melt Away! $500 Deposit Special (A-111)
Getting: Welcome Home to a Premiere Loft Community! (A-114)
Getting: Schedule a tour TODAY!
Getting: Do you use the bus to commute?
Getting: Check Anthony Mill Out Today!
Getting: $500 Security Deposit with good credit on 1 and 2 bedroom apartments
Getting: 2 Bedroom - HUGE walk in closet #A109
Getting: Beaut

Getting: Special Offer Going On Right Now!
Getting: Harris Mill Lofts
Getting: Special Offer Going On Right Now!
Getting: Come Take A Tour Today
Getting: Schedule a tour of a loft today!
Getting: Contact Harris Mill Today
Getting: New Year, New Apartment!
Getting: 2 Bedrooms and 1 Bathroom For Lease At Harris Mill Lofts
Getting: Contact Harris Mill Today
Getting: New Availability At Harris Mill
Getting: New Special Going On At Harris Mill, Right Now!
Getting: 2 Bedrooms and 1 Bathroom For Lease At Harris Mill Lofts
Getting: Contact Harris Mill Today
Getting: 2 Bedrooms and 1 Bathroom For Lease At Harris Mill Lofts
Getting: Schedule a tour of a loft today!
Getting: Contact Harris Mill Today
Getting: Special Offer Going On Right Now!
Getting: New Year, New Apartment!
Getting: 1 Bedroom for $1,235.00
Getting: Special Offer Going On Right Now!
Getting: New Availability At Harris Mill
Getting: 2 Bedrooms and 1 Bathroom For Lease At Harris Mill Lofts
Getting: Harris Mill Lofts - AVAILABILITY

...

Getting: Patio / balcony, Assigned parking, On-site laundry
Getting: Central a/c or window a/c, Clubhouse, Dog park
Getting: Vaulted ceilings, 66 acres of extensive lush landscaping, Dog park
Getting: Clubhouse, Townhouse floorplans, Outdoor swimming pool
Getting: Internet lounge, Clubhouse, Central a/c or window a/c
Getting: Clubhouse, Sports court, Strength and cardio fitness center
Getting: On-site laundry, A finch living community, Largeclosets
Getting: Strength and cardio fitness center, Patio & Vaulted Ceilings
Getting: Dishwasher, Open layouts, Outdoor swimming pool
Getting: Playground, Close to transportation, Assigned parking
Getting: Patio, Large closets, Dishwasher
Getting: Internet lounge, Outdoor swimming pool, Dog park
Getting: Open Concept, Central Air, Dishwasher
Getting: 2 bedroom 1 and a half bath
Getting: First Floor One Bed Riverside
Getting: Room
Getting: Having trouble finding a Pet Friendly Rental?
Getting: Gorgeous 1 bed w/ Gas Included! New counter-tops and car

Getting: Top Floor Corner Unit at Pocasset Mill Lofts! 2 Bed 2 Bath #332
Getting: Dreaming of Springtime on Your Private Patio? Pocasset Mill Lofts #110
Getting: Furry Friends are Welcome at Pocasset Mill!
Getting: Schedule a tour TODAY!
Getting: $500 Will Hold Your 1 Bedroom at Pocasset Mill! Top Floor Apt! #306
Getting: HUGE 2 Bedroom with a den and a walk-in pantry !
Getting: Need Extra Space? This 2 bed has a Bonus Den at Pocasset Mill! #126
Getting: Welcome Home, We Are Waiting For YOU at Pocasset Mill Lofts!
Getting: $500 Security Deposit with Good Credit! Come See Us at Pocasset Mill!
Getting: Need Extra Space? This 1 bed has a Bonus Den at Pocasset Mill! #209
Getting: Quality Living - It Starts Here at Pocasset Mill Lofts! #222
Getting: Need Extra Space? This 2 bed has a Bonus Den at Pocasset Mill! #323
Getting: Dreaming of Springtime on Your Private Patio? Pocasset Mill Lofts #110
Getting: It's not too late to make Pocasset Mill Lofts your New Home! #325
Getting: Make Your New

Getting: Spacious 2BR,  Pet Friendly, H&HW Included, Pool&Fitness Center!
Getting: HUGE 1BR,  Pet Friendly, H&HW Included, Pool&Fitness Center!
Getting: Large 1BR, FREE First Month Rent, H&HW Included!
Getting: Spacious 1BR, Utilities Included, Pets Welcome, FREE RENT!
Getting: Large 1BR, FREE First Month Rent, H&HW Included!
Getting: HUGE 1BR,  Pet Friendly, H&HW Included, Pool&Fitness Center!
Getting: HUGE 1BR,  Pet Friendly, H&HW Included, Pool&Fitness Center!
Getting: Spacious 1BR, Utilities Included, Pets Welcome, FREE RENT!
Getting: HUGE 1BR,  Pet Friendly, H&HW Included, FREE RENT!
Getting: Spacious 2BR,  Pet Friendly, H&HW Included, Pool&Fitness Center!
Getting: HUGE 1BR,  Pet Friendly, H&HW Included, Pool&Fitness Center!
Getting: Beautiful Johnston Ap $1000  Monthly, 2nd fl (Johnston, Rhode Island)
Getting: Beautiful Johnston Ap $1000  Monthly, 2nd fl (Johnston, Rhode Island)
Getting: newly availible
Getting: 325 Cherry Hill Rd, Johnston, RI 02919
Getting: 325 Cherry Hill Rd, 

Getting: Pocasset Mill is the Purr-fect New Home for 2020! (#120)
Getting: Live a Better Life at Pocasset Mill! (#306)
Getting: Make Yourself at Home at Pocasset Mill! (#120)
Getting: 1st Floor 2 Bed 2 Bath Corner Unit at Pocasset Mill! #120
Getting: SPECIAL: $500 Security Deposit on ALL 1 or 2 Beds! Pocasset Mill!
Getting: Pocasset Mill is Just Right for You! (#120)
Getting: Christmas Special - $500 Security Deposit!! Gift Yourself a New Home!
Getting: Dreaming of Springtime on Your Private Patio? Pocasset Mill Lofts #107
Getting: 1st Floor 2 Bed 2 Bath with a Bonus Den at Pocasset Mill Lofts! #126
Getting: Enjoy the Holidays in Your New Apartment at Pocasset Mill Lofts!
Getting: Dreaming of Springtime on Your Private Patio? Pocasset Mill Lofts #107
Getting: Top Floor 2 Bed 2 Bath Corner Unit at Pocasset Mill! #320
Getting: Welcome Home, We Are Waiting For YOU at Pocasset Mill Lofts!
Getting: Industrial Elegance Meets Modern Amenities at Pocasset Mill (#306)
Getting: Beautiful 2 Bedro

Getting: Top floor 2BR with fireplace and granite counters! Cranston
Getting: Chill out this winter and rent in Cranston!
Getting: Sit by the crackling fire in Cranston! Top floor 2BR available!
Getting: Upgraded FIRST FLOOR one bedroom! Cranston
Getting: It's snow wonder you'll love our apartments!
Getting: Leaf your worries behind, rent in Cranston!
Getting: Spacious, Pet Friendly, Laundry in Home, GREAT Location!
Getting: 2 Bedroom Apartment In Western Cranston! In Home Laundry, Heat Incl!
Getting: 1 Bedroom Apartment in Western Cranston! Available NOW!
Getting: 2 Bedroom in Cranston! MOVE IN ASAP, GET A $500 GIFT CARD!!!
Getting: NEW YEAR, NEW HOME! 2 Bedroom in Western Cranston
Getting: SPECIAL!! TOUR TODAY!
Getting: 1 Bed With In Home Laundry! MOVE IN ASAP!
Getting: 2 Bedroom Available! Receive a Gift When You Move In!
Getting: 2 BEDROOM SPECIAL RUNNING NOW! TOUR TODAY, MOVE IN NEXT WEEK!
Getting: SPECIAL $500 GIFT WHEN YOU RENT A 2 BEDROOM AT WESTERN HILLS! CALL NOW
Getting: $50